In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.5 MB/s eta 0:00:00


In [ ]:
import pdfplumber
import pandas as pd
import re

# Path to your PDF file
pdf_path = "/your file.pdf"  # Adjust to your file path

# List to hold all data rows
all_data = []
headers = None

# Open the PDF and process pages starting from page 2
with pdfplumber.open(pdf_path) as pdf:
    total_pages = len(pdf.pages)
    for page_num in range(1, total_pages):  # Page numbers start at 0, so 1 = page 2
        page = pdf.pages[page_num]
        # Extract tables from the page
        tables = page.extract_tables()

        for table in tables:
            if not table:
                continue

            # Convert table to DataFrame
            df = pd.DataFrame(table)

            # Remove unwanted text rows
            unwanted_phrases = [
                r"Page \d+ of \d+",  # Matches "Page X of Y"
                "This is a system generated statement issued through Mashreq Online"
            ]
            for phrase in unwanted_phrases:
                df = df[~df.apply(lambda row: any(re.search(phrase, str(cell)) for cell in row), axis=1)]

            if df.empty:
                continue

            # Extract headers from the first table (page 2)
            if headers is None:
                # Assume first two rows are headers, combine them
                headers = df.iloc[0:2].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=0).tolist()
                data = df.iloc[2:]  # Data starts after headers
                if not data.empty:
                    all_data.append(data)
            else:
                # Check if this table has headers repeated
                potential_headers = df.iloc[0:2].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=0).tolist()
                if potential_headers == headers:
                    data = df.iloc[2:]  # Skip headers
                    if not data.empty:
                        all_data.append(data)
                else:
                    # Assume entire table is data if headers don’t match
                    all_data.append(df)

# Combine all data into one DataFrame
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    final_df.columns = headers

    # Ensure numeric columns remain as strings to preserve decimals
    # No conversion to float, keep as raw strings
    # If you need to verify, uncomment below to see raw values
    # print(final_df[['Amount لمبلغ', 'Balance الرصيد']].head())

    # Save to Excel without float formatting to preserve original strings
    final_df.to_excel("output_file.xlsx", index=False, engine='openpyxl')
    print("Data saved to output_file.xlsx with original decimal precision")

    # Save to CSV without float formatting
    final_df.to_csv("output_file.csv", index=False)
    print("Data saved to output_file.csv with original decimal precision")
else:
    print("No tables found after cleaning.")

Data saved to output_file.xlsx with original decimal precision
Data saved to output_file.csv with original decimal precision
